In [40]:
import os
import random
from datetime import datetime
import csv
import pandas as pd
from sklearn.linear_model import LinearRegression

# Create output directory
output_dir = "sensor_data/csv"
os.makedirs(output_dir, exist_ok=True)

# User input for number of times to run
num_iterations = int(input("Enter number of air quality readings to generate: "))

# Generating timestamps with a one minute period
timestamps = pd.date_range(start='2025-01-01', periods=num_iterations, freq='1min')

In [41]:
# Function to generate random air quality data
def generate_air_quality_data(index, sensor_id=3):

    ts = timestamps[index].to_pydatetime().isoformat()

    nh3 = round(random.uniform(0, 10), 2)
    h2s = round(random.uniform(0, 0.01), 4)
    voc = round(random.uniform(50, 600), 2)
    no2 = round(random.uniform(0, 300), 2) if random.random() > 0.2 else None   # approx. 20% there are no no2 values

    toilet_environment = random.choice(["Fresh", "Noticeable", "Stinky"])

    return {
        "sensor_id": sensor_id,
        "timestamp": ts,
        "nh3": nh3,
        "h2s": h2s,
        "voc": voc,
        "no2": no2,
        "health_environment": toilet_environment
    }

In [42]:
# Impute only no2 values using mean, median, mode or regression
def impute_missing_values(df, strategy='mean'):
    if strategy == 'mean':
        df['no2'] = df['no2'].fillna(df['no2'].mean()).round(2)
    elif strategy == 'median':
         df['no2'] = df['no2'].fillna(df['no2'].median()).round(2)
    elif strategy == 'mode':
        df['no2'] = df['no2'].fillna(df['no2'].mode()).round(2)
    #elif strategy == 'regression':             Haben keinen direkten Zusammenhang zwischen den Werten weshalb des net funktioniert
        # Only impute no2 values using so2 as predictor
    #    hum_train = df.dropna(subset=['no2'])
    #    model = LinearRegression()
    #    model.fit(hum_train[['so2']], hum_train['no2'])
    #    hum_missing = df[df['no2'].isnull()]
    #    df.loc[df['no2'].isnull(), 'no2'] = model.predict(hum_missing[['so2']]).round(2)
    return df

Daten in CSV Speichern

In [43]:
# Function to save data as CSV
def save_air_quality_csv(filename, data):
    if not data:
        return
    filepath = os.path.join(output_dir, filename)
    with open(filepath, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

    print(f"Saved Air Quality CSV: {filepath}")

# Generate multiple batches of air quality data
readings = []
for i in range(num_iterations):
    air_quality_data = generate_air_quality_data(i)
    print(air_quality_data)
    readings.append(air_quality_data)

    # Add a duplicate at every 3rd reading
    if i % 3 == 0 and i != 0:
        readings.append(air_quality_data)
        print(air_quality_data)

# Create DataFrame
df = pd.DataFrame(readings)

# Save raw data (with duplicates)
save_air_quality_csv("air_quality_readings_errors.csv", df.to_dict(orient='records'))

{'sensor_id': 3, 'timestamp': '2025-01-01T00:00:00', 'nh3': 2.74, 'h2s': 0.0051, 'voc': 537.63, 'no2': None, 'health_environment': 'Fresh'}
{'sensor_id': 3, 'timestamp': '2025-01-01T00:01:00', 'nh3': 7.46, 'h2s': 0.0078, 'voc': 544.98, 'no2': 52.74, 'health_environment': 'Stinky'}
{'sensor_id': 3, 'timestamp': '2025-01-01T00:02:00', 'nh3': 4.52, 'h2s': 0.0031, 'voc': 544.37, 'no2': 205.33, 'health_environment': 'Fresh'}
{'sensor_id': 3, 'timestamp': '2025-01-01T00:03:00', 'nh3': 7.82, 'h2s': 0.0092, 'voc': 460.81, 'no2': None, 'health_environment': 'Fresh'}
{'sensor_id': 3, 'timestamp': '2025-01-01T00:03:00', 'nh3': 7.82, 'h2s': 0.0092, 'voc': 460.81, 'no2': None, 'health_environment': 'Fresh'}
{'sensor_id': 3, 'timestamp': '2025-01-01T00:04:00', 'nh3': 8.66, 'h2s': 0.0051, 'voc': 188.49, 'no2': 131.4, 'health_environment': 'Stinky'}
{'sensor_id': 3, 'timestamp': '2025-01-01T00:05:00', 'nh3': 7.8, 'h2s': 0.0046, 'voc': 412.86, 'no2': 40.43, 'health_environment': 'Noticeable'}
{'sensor_

Doppelte Werte entfernen und Fehlende Ergänzen:

In [44]:
# Remove duplicated rows
df = df.drop_duplicates()

# Impute missing no2 values 
df = impute_missing_values(df, strategy="mean")  #besser, aber anfälliger für ausreißer

# Save cleaned data (without duplicates)
save_air_quality_csv("air_quality_readings.csv", df.to_dict(orient='records'))

#df.to_csv("sensor_data/csv/air_quality_reading.csv", index=False)

Saved Air Quality CSV: sensor_data/csv\air_quality_readings.csv
